In [34]:
from bs4 import BeautifulSoup
import requests
import json
import time

In [33]:
def log_init():
    with open('log.txt', 'w') as log:
            log.write('Beginning Log\n')
def log_write(text):
    with open('log.txt', 'a') as log:
            log.write(f'{text}\n')

In [3]:
with open('games.txt', 'r') as f:
    games = json.load(f)

In [4]:
games

{'ps4': {'Grand Theft Auto V': {'title': 'Grand Theft Auto V',
   'page': '/game/playstation-4/grand-theft-auto-v',
   'critic_score': '97',
   'user_score': '8.3'},
  'The Last of Us Remastered': {'title': 'The Last of Us Remastered',
   'page': '/game/playstation-4/the-last-of-us-remastered',
   'critic_score': '95',
   'user_score': '9.1'},
  'God of War': {'title': 'God of War',
   'page': '/game/playstation-4/god-of-war',
   'critic_score': '94',
   'user_score': '9.2'},
  'XCOM 2: War of the Chosen': {'title': 'XCOM 2: War of the Chosen',
   'page': '/game/playstation-4/xcom-2-war-of-the-chosen',
   'critic_score': '94',
   'user_score': '6.2'},
  'Persona 5': {'title': 'Persona 5',
   'page': '/game/playstation-4/persona-5',
   'critic_score': '93',
   'user_score': '9.1'},
  'Metal Gear Solid V: The Phantom Pain': {'title': 'Metal Gear Solid V: The Phantom Pain',
   'page': '/game/playstation-4/metal-gear-solid-v-the-phantom-pain',
   'critic_score': '93',
   'user_score': '8.2

In [24]:
sample_url = 'http://www.metacritic.com/game/playstation-4/life-is-strange/user-reviews?page=1'

In [25]:
headers = {'User-agent': 'Maxine .1'}
res = requests.get(sample_url, headers=headers)
print(f"Accessing page at {sample_url}")
print(res.status_code)
page_source = res.content

Accessing page at http://www.metacritic.com/game/playstation-4/life-is-strange/user-reviews?page=1
200


## ok so looking through the html, I see that ones that need to be expanded are in the html even without being expanded, so we don't need to click. However, I figured out how to click.  

* span class='blurb blurb_expanded'
* the issue is that reviews without a need for collapse don't have this span at all

In [26]:
soup = BeautifulSoup(page_source, 'html.parser')

In [27]:
reviews = soup.find_all('div', attrs={'class': 'review_body'})

## Here is a review that needed to be expanded

In [28]:
reviews[0]

<div class="review_body">
<strong class="bold">
            This review contains spoilers<span class="toggle_text_visibility">, click expand to view</span>.
        </strong>
<span class="inline_expand_collapse inline_collapsed" id="review_blurb_6435927"><span class="blurb blurb_collapsed"></span><span class="blurb blurb_expanded">This game is heart breaking
<br/>For me personally, i can't relate to the character until around episode 3 where it all starts to get good, and at that point on, this game was amazing.
<br/>It does not have the best start, Episode 1 is Ok, And Episode 2 is great, but the real moment of this game is episode 3 and 4, Extremely amazing
<br/> <br/>This game teaches us, that sometimes, what matter are the journey we go through.
<br/>A True Roller Coaster of feeling!</span><span class="blurb_etc">…</span> <a class="toggle_expand_collapse toggle_expand" href="/game/playstation-4/life-is-strange/user-reviews?page=1&amp;user_review_id=6435927" rel="nofollow">Expand</a

In [29]:
# This has quotes for some reason
reviews[0].find('span', attrs={'class': 'blurb_expanded'}).text

"This game is heart breaking\rFor me personally, i can't relate to the character until around episode 3 where it all starts to get good, and at that point on, this game was amazing.\rIt does not have the best start, Episode 1 is Ok, And Episode 2 is great, but the real moment of this game is episode 3 and 4, Extremely amazing\r This game teaches us, that sometimes, what matter are the journey we go through.\rA True Roller Coaster of feeling!"

## Here is a review that didn't need to be expanded

In [30]:
reviews[1]

<div class="review_body">
<span>I am a strong man, never have wanted to cry and and rip my heart out more tha when I played this game. I was so emotionally  invested  in a game Icared more for them than me. Artistic  and beautiful  beyond words. Am must play for all story  seeking gamers</span>
</div>

In [36]:
reviews[-1]

<div class="review_body">
                                Despite its obvious flaws the reactive story and compelling characters make this one of the best storytelling experiences of the current generation.
                            </div>

# Review scraping functions

In [26]:
def extract_reviews_by_console(console):
    start = time.time()
    log_init()
    
    reviews = []
    for title in games[console]:
        game = games[console][title]
        game['reviews'] = extract_reviews_from_game(game)
        reviews['title'] =  game
        log_write(f"Elapsed Time: {time.time() - start}")
        

In [36]:
def extract_reviews_from_game(game):
    review_list = []
    #generate url from game object
    url = "http://www.metacritic.com" + game['page'] + "/user-reviews"
    # get page request and soup object
    headers = {'User-agent': f'1.{game["title"]}'}
    res = requests.get(url, headers=headers)
    print(f"Accessing page at {url}")
    print(res.status_code)
    if(res.status_code > 300):
        log_write(f"Couldn't access game {game['GID']} : {game['title']}")
        return review_list
    
    page_source = res.content
    soup = BeautifulSoup(page_source, 'html.parser')
    try:
        num_pages = soup.find('li', attrs={'class': 'last_page'}).find('a').text
    except:
        num_pages = 1
    for i in range(int(num_pages)):
        new_url = f"{url}?page={i}"
        revs = extract_reviews_from_page(new_url)
        [rev.update({'GID': game['GID']}) for rev in revs]
        review_list.extend(revs)
    return review_list

In [28]:
def extract_reviews_from_page(url):
    
    review_list = []
    
    # get page request and soup object
    headers = {'User-agent': 'game["title"]'}
    res = requests.get(url, headers=headers)
    log_write(f"Accessing page at {url}")
    print(res.status_code)
    if(res.status_code > 300):
        log_write(f"Could not reach page: {url}")
        return
    
    page_source = res.content
    soup = BeautifulSoup(page_source, 'html.parser')
    
    #save a list of reviews
    try:
        reviews = soup.find('ol', attrs={'class' : 'user_reviews'}).find_all('div', attrs={'class': 'review_content'})
    except:
        log_write(f"No reviews found on page: {url}")
        return review_list
    # loop through all reviews:
    for review in reviews:
        # extract the text, the review score, and the author, and date
        try:
            review_list.append(extract_single_review(review))
        except: 
            print(review)
    return review_list
    
       

In [29]:
def extract_single_review(soup):
    review = {}
    review['author'] = soup.find('a').text
    review['date'] = soup.find('div', attrs={'class':'date'}).text
    review['score'] = soup.find('div', attrs={'class': 'metascore_w'}).text
    review['blurb'] = extract_text(soup.find('div', attrs={'class': 'review_body'}))
    return review

In [30]:
def extract_text(review):
    if(not review.find('span', attrs={'class': 'blurb_expanded'})):
        return review.find('span').text
    else:
        return review.find('span', attrs={'class': 'blurb_expanded'}).text

In [32]:
start = time.time
reviews = extract_reviews_by_console('ps4')
end = time.time - start

Accessing page at http://www.metacritic.com/game/playstation-4/grand-theft-auto-v/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/grand-theft-auto-v/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/grand-theft-auto-v/user-reviews?page=1
200
Accessing page at http://www.metacritic.com/game/playstation-4/grand-theft-auto-v/user-reviews?page=2
200
Accessing page at http://www.metacritic.com/game/playstation-4/grand-theft-auto-v/user-reviews?page=3
200
Accessing page at http://www.metacritic.com/game/playstation-4/grand-theft-auto-v/user-reviews?page=4
200
Accessing page at http://www.metacritic.com/game/playstation-4/the-last-of-us-remastered/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/the-last-of-us-remastered/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/the-last-of-us-remastered/user-reviews?page=1
200
Accessing page at http://www.metacritic

Accessing page at http://www.metacritic.com/game/playstation-4/bloodborne/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/bloodborne/user-reviews?page=1
200
Accessing page at http://www.metacritic.com/game/playstation-4/bloodborne/user-reviews?page=2
200
Accessing page at http://www.metacritic.com/game/playstation-4/bloodborne/user-reviews?page=3
200
Accessing page at http://www.metacritic.com/game/playstation-4/bloodborne/user-reviews?page=4
200
Accessing page at http://www.metacritic.com/game/playstation-4/bloodborne/user-reviews?page=5
200
Accessing page at http://www.metacritic.com/game/playstation-4/bloodborne/user-reviews?page=6
200
Accessing page at http://www.metacritic.com/game/playstation-4/bloodborne/user-reviews?page=7
200
Accessing page at http://www.metacritic.com/game/playstation-4/bloodborne/user-reviews?page=8
200
Accessing page at http://www.metacritic.com/game/playstation-4/bloodborne/user-reviews?page=9
200
Accessing page at ht

Accessing page at http://www.metacritic.com/game/playstation-4/dragon-age-inquisition/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/dragon-age-inquisition/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/dragon-age-inquisition/user-reviews?page=1
200
Accessing page at http://www.metacritic.com/game/playstation-4/dragon-age-inquisition/user-reviews?page=2
200
Accessing page at http://www.metacritic.com/game/playstation-4/dragon-age-inquisition/user-reviews?page=3
200
Accessing page at http://www.metacritic.com/game/playstation-4/dragon-age-inquisition/user-reviews?page=4
200
Accessing page at http://www.metacritic.com/game/playstation-4/battlefield-1/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/battlefield-1/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/battlefield-1/user-reviews?page=1
200
Accessing page at http://www.metacritic.com/game/pl

Accessing page at http://www.metacritic.com/game/playstation-4/dragon-ball-fighterz/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/batman-arkham-knight/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/batman-arkham-knight/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/batman-arkham-knight/user-reviews?page=1
200
Accessing page at http://www.metacritic.com/game/playstation-4/batman-arkham-knight/user-reviews?page=2
200
Accessing page at http://www.metacritic.com/game/playstation-4/batman-arkham-knight/user-reviews?page=3
200
Accessing page at http://www.metacritic.com/game/playstation-4/batman-arkham-knight/user-reviews?page=4
200
Accessing page at http://www.metacritic.com/game/playstation-4/batman-arkham-knight/user-reviews?page=5
200
Accessing page at http://www.metacritic.com/game/playstation-4/batman-arkham-knight/user-reviews?page=6
200
Accessing page at http://www.metacr

Accessing page at http://www.metacritic.com/game/playstation-4/final-fantasy-xiv-heavensward/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/final-fantasy-xiv-online-a-realm-reborn/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/final-fantasy-xiv-online-a-realm-reborn/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/dark-souls-remastered/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/dark-souls-remastered/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/destiny-the-taken-king/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/destiny-the-taken-king/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/destiny-the-taken-king/user-reviews?page=1
200
Accessing page at http://www.metacritic.com/game/playstation-4/destiny-the-taken-king/user-reviews?pag

...

Accessing page at http://www.metacritic.com/game/playstation-4/frantics/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/frantics/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/mirrors-edge-catalyst/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/mirrors-edge-catalyst/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/octodad-dadliest-catch/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/octodad-dadliest-catch/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/the-bunker/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/the-bunker/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/dynasty-warriors-8-xtreme-legends-complete-edition/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/dynasty-

Accessing page at http://www.metacritic.com/game/playstation-4/dead-synchronicity-tomorrow-comes-today/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/dead-synchronicity-tomorrow-comes-today/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/its-quiz-time/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/its-quiz-time/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/wwe-2k17/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/wwe-2k17/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/kings-quest-chapter-2-rubble-without-a-cause/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/kings-quest-chapter-2-rubble-without-a-cause/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/call-of-duty-black-ops-iii---descent/user-reviews
200

Accessing page at http://www.metacritic.com/game/playstation-4/the-25th-ward-the-silver-case/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/vesta/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/vesta/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/dear-esther-landmark-edition/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/dear-esther-landmark-edition/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/io/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/io/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/light-tracer/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/light-tracer/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/root-letter/user-reviews
200
Accessing page at htt

Accessing page at http://www.metacritic.com/game/playstation-4/motogp-18/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/nights-of-azure/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/nights-of-azure/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/how-we-soar/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/how-we-soar/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/nine-parchments/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/nine-parchments/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstation-4/batman-the-enemy-within---episode-2-the-pact/user-reviews
200
Accessing page at http://www.metacritic.com/game/playstation-4/batman-the-enemy-within---episode-2-the-pact/user-reviews?page=0
200
Accessing page at http://www.metacritic.com/game/playstat

TypeError: 'NoneType' object is not iterable

In [ ]:
end

In [ ]:
with open('reviews.txt', 'w') as f:
    json.dump(reviews, f)